يهدف هذا المشروع إلى تطوير نظام ذكي للكشف عن الأجسام باستخدام نموذج YOLOv5 المدعوم بـ PyTorch، مع توظيف كاميرا الهاتف المحمول كمصدر للرؤية الحاسوبية عبر بروتوكول الشبكة. تم إعداد بيئة Python 3.10 مستقلة خصيصًا لهذا المشروع على نظام Ubuntu لضمان سهولة النقل إلى أجهزة مختلفة، خاصةً أجهزة Raspberry Pi 5 التي تمتاز بمواصفات قوية.

تم الاستفادة من تطبيق IP Webcam لربط الهاتف بكاميرا الحاسوب عبر الاتصال بشبكة USB Tethering، مما يضمن اتصالًا سريعًا ومستقرًا بدون الحاجة إلى Wi-Fi. يتم استقبال بث الفيديو في البرنامج باستخدام مكتبة OpenCV، ثم يتم تمرير الإطارات إلى نموذج YOLOv5 لتحليلها واستخراج الكائنات الظاهرة.

يدعم المشروع:

    التشغيل على بيئات Linux بسهولة.

    الاعتماد الكامل على وحدة المعالجة الرسومية (GPU) في حال توفرها.

    التوسع مستقبلًا ليصبح نظام API قادر على استقبال الصور من أجهزة مختلفة.

يمثل هذا المشروع قاعدة مرنة لأنظمة الرؤية الحاسوبية، ويمكن توظيفه لاحقًا في تطبيقات مثل التتبع الذكي، الطائرات المسيّرة، أو أنظمة الأمان الذكية.

In [ ]:
import cv2
import torch

يستورد مكتبة OpenCV التي تُستخدم لقراءة الفيديو من الكاميرا (سواء IP أو USB)، ومعالجة الإطارات (Frames)، وعرض النتائج على الشاشة.

بدونها ما تقدر تتعامل مع الكاميرا ولا تعرض المخرجات.

يستورد مكتبة PyTorch وهي الأساس في تشغيل نموذج YOLOv5.



In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  
model.eval()

Using cache found in /home/alhassan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-9 Python-3.10.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

| رقم الطبقة       | اسم الطبقة         | الوظيفة                                                                            |
| ---------------- | ------------------ | ---------------------------------------------------------------------------------- |
| (0)              | `Conv2d(3, 32)`    | أول طبقة، تحول الصورة من 3 قنوات (RGB) إلى 32 خريطة ملامح (feature maps).          |
| (2), (4), (6)... | `C3`, `Bottleneck` | طبقات ذكية تجمع الملامح من الصورة بأشكال مختلفة، وتُستخدم لتقليل الفقد بالمعلومات. |
| (9)              | `SPPF`             | تجميع ملامح متعدد المقاييس (سريع وأذكى من SPP العادي).                             |
| (11), (15)       | `Upsample`         | لإعادة تكبير الصورة حتى ندمج الملامح منخفضة الدقة مع العالية.                      |
| (24)             | `Detect`           | آخر طبقة مسؤولة عن **الكشف (bounding boxes + تصنيفات)**.                           |


In [ ]:
ip_webcam_url = "http://192.168.1.101:8080/video"
cap = cv2.VideoCapture(ip_webcam_url)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    results = model(frame)

    results.render()  
    cv2.imshow("Detections", results.ims[0])

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

/home/alhassan/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/alhassan/py310base/vscodekernel/lib/python3.10/site-packages/cv2/qt/plugins"
/home/alhassan/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/alhassan/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/alhassan/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Ple

فتح نافذة فيديو مباشرة من كاميرا الهاتف.

الكشف عن الأشياء (شخص، كرسي، هاتف...) في الوقت الحقيقي.

المربعات (Bounding Boxes) تظهر تلقائيًا مع اسم التصنيف ونسبة الدقة.

In [ ]:
cap.release()
cv2.destroyAllWindows()

تغلق الاتصال بين OpenCV و مصدر الكاميرا (IP أو USB).

تمنع الكاميرا من أن تبقى محجوزة بعد انتهاء البرنامج.

إذا نسيتها، ممكن الكاميرا تظل شغالة بالخلفية ولا تعمل مره ثانية إلا بعد إعادة تشغيل النظام أو قتل العملية يدويًا.

لمنع تراكم نوافذ عرض الصور.

ولضمان إغلاق البرنامج بدون مشاكل رسومية (خاصة في Linux).